# Analisis de cuentas de un banco ficticio para saber si los clientes van a abandonar el banco o no

# Part 1: Data Preprocessing

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing datasets
Las primeras 3 columnas son numero de columna, id de cliente, y nombre de cliente, por lo que no nos interesan.
La ultima columna dice si dejó o no dejó el banco después de 6 meses

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

## Encoding the categorical variables
Enumera el nombre del país y el genero, y no me acuerdo que hace el hot encoder

In [27]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:,2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
X.shape

(10000, 12)

## Splitting the dataset into the Training set and Test set

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train.shape

(8000, 12)

## Feature scaling
ACP

In [32]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train.shape

(8000, 12)

# Part 2: Now let's make the ANN!

## Importing the Keras libraries and packages

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense

## Initializing the ANN

In [38]:
classifier = Sequential()

## Adding the input layer and the first hidden layer
6 neuronas, inicializa los pesos con una distribucion uniforme, relus, 12 entradas

In [39]:
classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))

## Adding the second hidden layer
6 neuronas, uniforme, relu

In [40]:
classifier.add(Dense(6, kernel_initializer = 'uniform', activation = 'relu'))

## Adding the output layer
1 neurona, uniforme, binaria

In [41]:
classifier.add(Dense(1, kernel_initializer = 'uniform', activation = 'sigmoid'))

## Compiling the ANN
b-prop

In [42]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Fitting the ANN to the Training set

In [43]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 3s 380us/step - loss: 0.4892 - acc: 0.7955
Epoch 2/100
8000/8000 [==============================] - 2s 293us/step - loss: 0.4295 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 294us/step - loss: 0.4247 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 293us/step - loss: 0.4204 - acc: 0.8075
Epoch 5/100
8000/8000 [==============================] - 2s 299us/step - loss: 0.4173 - acc: 0.8247
Epoch 6/100
8000/8000 [==============================] - 2s 294us/step - loss: 0.4150 - acc: 0.8285
Epoch 7/100
8000/8000 [==============================] - 2s 297us/step - loss: 0.4138 - acc: 0.8292
Epoch 8/100
8000/8000 [==============================] - 2s 297us/step - loss: 0.4120 - acc: 0.8319
Epoch 9/100
8000/8000 [==============================] - 2s 304us/step - loss: 0.4106 - acc: 0.8327
Epoch 10/100
8000/8000 [==============================] - 2s 303us/step - loss: 0.4101 - acc: 0.8339

# Part 3: Making the predictions and evaluating the model

## Predicting the Test set results

In [51]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5) # tupla de booleanos

## Making the Confusion Matrix

In [52]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [53]:
cm

array([[1545,   50],
       [ 263,  142]])

## Predicting a single new observation
credit score: 600
geography: france
age: 40
tenure: 3
balance: 60000
number of products: 2
has credit card: yes
is active member: yes
estimated salary: 50000

In [79]:
homework_client = np.array([[0, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])
homework_client = sc.transform(homework_client)
new_prediction = classifier.predict(homework_client)
print(new_prediction)

[[ 0.14901029]]


/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
